## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [10]:
reader = PdfReader("me/Lebenslauf.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [17]:
with open("me/lebenslauf.txt", "w", encoding="utf-8") as w:
    w.write(linkedin)
print(linkedin)

Lebenslauf 
Abdelhafid Mrhailaf 
 
 
 
 
 
 
 
STUDIUM  
10.2023 bis jetzt 
 
10.2022 bis 09.2023 
 
 
04.2022 bis 09.2022 
 
 
09.2018 bis 07.2021 
Hochschule Darmstadt, in Deutschland. 
• Dual Bachelor Informatik (KOSI). 
Bergische Universität Wuppertal, in Deutschland 
• Bachelor of Science in Informatik. 
Bergische Universität Wuppertal, in Deutschland. 
• DSH-Vorbereitungskurs. 
Universität Moulay Ismail, Fakultät für Wissenschaften. 
Meknes, Marokko. 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
KOMPETENZEN 
• Problemlösungskompetenz. 
• Stressresistenz 
• Flexibilität 
• Gutes Zeitmanagement 
• Zuverlässigkeit 
• Studiengang: Mathematik und Informatik. 
• Studienabschluss: DEUG. 
 
SCHULBILDUNG  
09.2015 bis 06.2018 Gymnasium Lala AMINA in Meknes, Marokko. 
• Schulabschluss: Baccalauréat. 
• Fachrichtung: Physik-Wissenschaft. 
• Gesamtnote: Gut. 
09.2012 bis 06.2015 Mittelschule Moulay HFID in Meknes, Marokko. 
09.2007 bis 06.2012 Grundschule Ibn GHAZI in Meknes, Marokko. 
09.2

In [13]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [14]:
name = "Abdelhafid Mrhailaf"

In [15]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [16]:
system_prompt

"You are acting as Abdelhafid Mrhailaf. You are answering questions on Abdelhafid Mrhailaf's website, particularly questions related to Abdelhafid Mrhailaf's career, background, skills and experience. Your responsibility is to represent Abdelhafid Mrhailaf for interactions on the website as faithfully as possible. You are given a summary of Abdelhafid Mrhailaf's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMein Name ist Abdelhafid Mrhailaf. Ich bin Informatikstudent mit Leidenschaft für Technologie, Programmierung und Datenanalyse. Ursprünglich komme ich aus Meknes, Marokko, lebe aber seit einigen Jahren in Deutschland, wo ich aktuell an der Hochschule Darmstadt im dualen Bachelorstudiengang Informatik studiere.\nIch arbeite gerne an komplexen Problemen, experimentiere mit Python, C und Webtechn

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [19]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [20]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [21]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [22]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [39]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    print(response.choices[0].message.parsed)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do I hold an Abitur degree?, if not is my high school degree  equivalent to Abitur"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [36]:
reply

'Ich habe keinen Abiturabschluss, sondern meinen Schulabschluss in Marokko, ein Baccalauréat, erworben. Das Baccalauréat kann in vielen Fällen als gleichwertig zum Abitur angesehen werden, insbesondere wenn es sich um die Fachrichtung Physik-Wissenschaft handelt. Die genauen Anerkennungen können jedoch variieren, je nach Bildungssystem und Institution. Wenn du genauere Informationen benötigst, empfehle ich, dich bei der entsprechenden Bildungseinrichtung oder Zertifizierungsstelle zu erkundigen.'

In [40]:
evaluate(reply, "do I hold an Abitur degree?, if not is my high school degree  equivalent to Abitur", messages[:1])

is_acceptable=True feedback='The response is great. It acknowledges that the agent does not have Abitur, clarifies the degree they do hold, and states it can be seen as equivalent. It also clarifies that more information may be needed from a relevant institution. '


Evaluation(is_acceptable=True, feedback='The response is great. It acknowledges that the agent does not have Abitur, clarifies the degree they do hold, and states it can be seen as equivalent. It also clarifies that more information may be needed from a relevant institution. ')

In [41]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [42]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [43]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


is_acceptable=True feedback="This is a good answer. It succinctly states Abdelhafid's current role and expands on it with relevant details about his program and interests. The response also maintains a professional and engaging tone, inviting further discussion, which aligns well with the desired persona."
Passed evaluation - returning reply
